In [42]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
'''This object details the dynamics of a an agent(vehicle) in the simulation'''
class Vehicle:
    def __init__(self, position, speed, max_speed, vehicle_length, vehicle_width, braking_prob):
        '''attributes of the vehicle'''
        self.position = position #position of the vehicle
        self.speed = speed #speed of the vehicle
        self.max_speed = max_speed #maximum speed of the vehicle
        self.vehicle_length = vehicle_length 
        self.vehicle_width = vehicle_width
        self.braking_prob = braking_prob

    def accelerate(self):
        if self.speed < self.max_speed:
            self.speed += 1

    def decelerate(self, headway_distance):
        if self.speed > headway_distance:
            self.speed = headway_distance #reduce speed equivalent to headay distance

    def random_brake(self):
        if np.random.rand() < self.braking_prob and self.speed > 0:
            self.speed -= 1

In [51]:
class Road:
    def __init__(self, length, width):
        self.length = length
        self.width = width
        self.road_state = np.zeros((self.length, self.width), dtype=int)

    def update_road_state(self, new_state):
        self.road_state = new_state

    def get_road_state(self):
        return self.road_state

In [56]:
class Controller:
    def __init__(self, road, production_prob, max_speed, braking_prob):
        self.road = road
        self.production_prob = production_prob
        self.max_speed = max_speed
        self.braking_prob = braking_prob
        #self.vehicles = [] #array of vehicles

    def produce_vehicles(self):
        '''This method aims to determine when to produce vehicles. If it's time to produce,
        vehicle dimensions are chosen. 
        1x1 for Motocycle, 2x2 for Car, 5x2 for small trucks, 9x2 for long trucks'''
        if np.random.rand() < self.production_prob:
            vehicle_length = np.random.choice([1, 2, 5, 9]) 
            if vehicle_length == 1:
                vehicle_width = 1
            else:
                vehicle_width = 2
            if np.sum(self.road.road_state[0:vehicle_length, 0:vehicle_width]) == 0: #Checks if the beginning of the road is available for the vehicle
                new_vehicle = Vehicle(0, np.random.randint(1, self.max_speed + 1), self.max_speed, vehicle_length, vehicle_width, self.braking_prob==0.1)
                return new_vehicle
                #self.vehicles.append((i, j, new_vehicle))
                #self.road.road_state[0:new_vehicle.vehicle_length, 0:new_vehicle.vehicle_width] = 1
        return None

    def move_vehicles(self, vehicle):
        vehicle.accelerate()
        vehicle.decelerate(min(self.road.length, vehicle.speed))
        vehicle.random_brake()
        
    def remove_vehicles(self, vehicle):
        if vehicle.position + vehicle.vehicle_length >=self.road.length:
            self.road.road_state[vehicle.position:vehicle.position+vehicle.vehicle_length, 0:vehicle.vehicle_width]=0
            return True
        return False
        #self.road.road_state = np.roll(self.road.road_state, -1, axis=0)
        #self.road.road_state[-1, :] = 0


    def update_road_state(self):
        

In [55]:
def visualize_simulation(road, controller, T):
    road_states = []
    for t in range(T):
        controller.produce_vehicles()
        controller.move_vehicles()
        controller.remove_vehicles()
        road_states.append(road.get_road_state().copy())

    road_states = np.array(road_states)
    plt.figure(figsize=(10, 6))
    plt.imshow(road_states, cmap='binary', origin='lower', aspect='auto')
    plt.colorbar(label='Vehicle')
    plt.xlabel('Position')
    plt.ylabel('Time step')
    plt.title('Spatiotemporal plot of traffic simulation')
    plt.show()


# Parameters
length = 50  # Length of the road
width = 2  # Width of the road
production_prob = 0.3  # Probability of producing vehicles
max_speed = 5  # Maximum speed of vehicles
braking_prob = 0.05  # Probability of random braking
T = 20  # Number of timesteps

# Initialization
road = Road(length, width)
controller = Controller(road, production_prob, max_speed, braking_prob)

visualize_simulation(road, controller, T)

NameError: name 'i' is not defined